Este módulo obtiene N cantidad de cajeros de cualquier tipo de los 4 pre-cluster definidos.

In [11]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../../Insumos/df_general.parquet')

In [12]:
def construir_resumen_estructural(df):
    
    resumen = (
        df
        .groupby("cajero")
        .agg(
            dias_obs=("retiro", "count"),
            mediana=("retiro", "median"),
            p95=("retiro", lambda x: x.quantile(0.95))
        )
        .reset_index()
    )
    
    resumen["ratio_p95_mediana"] = (
        resumen["p95"] / resumen["mediana"]
    )
    
    resumen["ratio_p95_mediana"] = resumen["ratio_p95_mediana"].replace([np.inf, -np.inf], 0)
    resumen["ratio_p95_mediana"] = resumen["ratio_p95_mediana"].fillna(0)
    
    return resumen

In [13]:
def aplicar_reglas_precluster(
    df_resumen,
    umbral_grande,
    umbral_estable,
    umbral_evento,
    umbral_picos
):
    
    def clasificar(row):
        mediana = row["mediana"]
        ratio = row["ratio_p95_mediana"]
        
        if mediana == 0:
            return "event_driven"
        elif (mediana >= umbral_grande) and (ratio <= umbral_estable):
            return "grande_y_estable"
        elif ratio >= umbral_evento:
            return "event_driven"
        elif ratio >= umbral_picos:
            return "normal_con_picos"
        else:
            return "normal_estable"
    
    df_resumen["etiqueta"] = df_resumen.apply(clasificar, axis=1)
    
    return df_resumen

In [14]:
resumen = construir_resumen_estructural(df)

df_clasificados = aplicar_reglas_precluster(
    resumen,
    umbral_grande=200000,
    umbral_estable=3,
    umbral_evento=8,
    umbral_picos=5
)

In [ ]:
def seleccionar_cajeros_por_cluster(
    df_clasificados,
    cluster,
    n,
    modo="aleatorio",
    random_state=42
):
    
    df_cluster = df_clasificados[
        df_clasificados["etiqueta"] == cluster
    ].copy()
    
    total_disponibles = len(df_cluster)
    
    if total_disponibles == 0:
        raise ValueError(f"No existen cajeros en el cluster '{cluster}'")
    
    if n > total_disponibles:
        print(f"Solo hay {total_disponibles} cajeros en '{cluster}'. Se devolverán todos.")
        n = total_disponibles
    
    if modo == "aleatorio":
        seleccion = df_cluster.sample(n=n, random_state=random_state)
        
    elif modo == "mayor_ratio":
        seleccion = df_cluster.sort_values(
            "ratio_p95_mediana",
            ascending=False
        ).head(n)
        
    elif modo == "mayor_mediana":
        seleccion = df_cluster.sort_values(
            "mediana",
            ascending=False
        ).head(n)
        
    else:
        raise ValueError("Modo no válido.")
    
    return seleccion.reset_index(drop=True)

In [ ]:
seleccionar_cajeros_por_cluster(
    df_clasificados,
    cluster="event_driven",
    n=5,
    modo="mayor_ratio"
)


'''Las 4 categorías que existen son:

    - event_driven
    - grande_y_estable
    - normal_con_picos
    - normal_estable
'''

#NOTA: Sería mejor si la columna etiqueta estuviera junto a cajero. 

,cajero,dias_obs,mediana,p95,ratio_p95_mediana,etiqueta
0,JF001931,754,50.0,108760.0,2175.200000,event_driven
1,JF000995,754,350.0,209530.0,598.657143,event_driven
2,JF001356,754,1600.0,329355.0,205.846875,event_driven
3,JF002402,754,7850.0,862425.0,109.863057,event_driven
4,JF001124,754,3050.0,170250.0,55.819672,event_driven
